In [ ]:
%reload_ext autoreload
%autoreload 2

import vtk
from vtk.util import numpy_support

import numpy as np
import bloscpack as bp

import sys
import os
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from annotation_utilities import *
from metadata import *

import time

import matplotlib.pyplot as plt
%matplotlib inline

from vis3d_utilities import *

In [38]:
stack = 'MD589'

In [39]:
name_to_label = DataManager.load_annotation_volume_nameToLabel(stack=stack, downscale=32)
label_to_name = {l: n for n, l in name_to_label.iteritems()}

In [43]:
vol_xmin, vol_xmax, vol_ymin, vol_ymax, vol_zmin, vol_zmax = \
DataManager.load_volume_bbox(stack=stack, type='annotation', downscale=32)

print 'annotation volume bbox :', vol_xmin, vol_xmax, vol_ymin, vol_ymax, vol_zmin, vol_zmax

annotation volume bbox : 622 1042 155 502 127 503


In [ ]:
colors = {name_u: np.array(random_colors(1)[0])/255.
          for name_u in set([convert_name_to_unsided(name_s) for name_s in name_to_label.keys()])}

In [ ]:
# with open('colors.txt', 'r') as f:
#     colors = {l[0]: np.r_[float(l[1]), float(l[2]), float(l[3])] for l in map(lambda x: x.split(), f.readlines())}

In [ ]:
# Render structures

In [ ]:
volume = bp.unpack_ndarray_file(DataManager.get_annotation_volume_filepath(stack=stack, downscale=32))

In [ ]:
for name_s, label in name_to_label.iteritems():
    
    xmin, xmax, ymin, ymax, zmin, zmax = bbox_3d(volume == label) 
    
    print name_s, xmin, xmax, ymin, ymax, zmin, zmax
    
    vol = (volume[ymin:ymax+1, xmin:xmax+1, zmin:zmax+1] == label).astype(np.float)

    t = time.time()

    polydata = volume_to_polydata(vol, origin=(vol_xmin+xmin,vol_ymin+ymin,vol_zmin+zmin), num_simplify_iter=3, smooth=True)
    # coordinates with respect to aligned uncropped volume

    sys.stderr.write('time: %.2f\n' % (time.time() - t) ) # ~ 150s for outerContour; 93s for sp5

    save_mesh_stl(polydata, os.path.join(MESH_ROOTDIR, 
                stack, "%(stack)s_down32_annotationVolume_%(name)s_smoothed.stl" % {'stack': stack, 'name': name_s}))

In [ ]:
# name_to_label_hindbrain = {name: label for name, label in name_to_label.iteritems() 
#                            if convert_name_to_unsided(name) in ['Pn', 'VLL', 'RtTg', 'RMC', '7n', '7N', '5N', '10N', '12N', 'AP', 'LRt', 'Amb', '6N', 'Tz']}

In [ ]:
polydata_list = {}

# for name_s, label in name_to_label_hindbrain.iteritems():
for name_s, label in name_to_label.iteritems():
    
    fn = os.path.join(MESH_ROOTDIR, stack, "%(stack)s_down32_annotationVolume_%(name)s_smoothed.stl" % {'stack': stack, 'name': name_s})
    polydata_list[name_s] = load_mesh_stl(fn, return_polydata_only=True)

mesh_actors = [actor_mesh(polydata, colors[convert_name_to_unsided(name_s)], wireframe=False, opacity=.5) 
               for name_s, polydata in polydata_list.iteritems()]

In [ ]:
fn = os.path.join(MESH_ROOTDIR, stack, "%(stack)s_down32_outerContourVolume_smoothed.stl" % {'stack': stack})
outerContour_polydata = load_mesh_stl(fn, return_polydata_only=True)

outerContour_actor = actor_mesh(outerContour_polydata, (1,1,1), opacity=.1, wireframe=False)

In [ ]:
launch_vtk(mesh_actors + [outerContour_actor], init_angle='30', background_color=(0,0,0))

In [ ]:
# Export to rgb PLY file (STL file can not have color info)
# https://github.com/Kitware/VTK/blob/master/IO/PLY/Testing/Python/TestPLYReadWrite.py#L31
# http://www.vtk.org/doc/nightly/html/classvtkPLYWriter.html#aa7f0bdbb2decdc7a7360a890a6c10e8b

for name_s, polydata in polydata_list.iteritems():
    fn = os.path.join(mesh_rootdir, stack, "%(stack)s_down32_%(name)s_smoothed.ply" % {'stack': stack, 'name':name_s})
    save_mesh(polydata, fn, color=(colors[convert_name_to_unsided(name_s)]*255).astype(np.int))